In [ ]:
import socket
import time

In [2]:
from enuActor.Controllers.bufferedSocket import BufferedSocket
from enuActor.Controllers.device import Device

In [4]:
TCP_IP = 'biasha1'
TCP_PORT = 23

dev = Device(TCP_IP, TCP_PORT)

In [5]:
def processSeq(sequence):
    for cmd, resp in sequence:
        print cmd
        r = dev.sendOneCommand(cmd)
        print r, "==", resp
        if r!=resp:
            raise ValueError
        time.sleep(0.5)
        
def testAll(state):
    allCmds = ["bia_on", 
           "bia_off",
           "shut_open", 
           "shut_close",
           "blue_open", 
           "blue_close",
           "red_open", 
           "red_close",
           "init"]

    for cmd in allCmds:
        print "cmd=",cmd
        processSeq(state)
        ret = dev.sendOneCommand(cmd)
        stat = dev.sendOneCommand("status")

        print "ret=",ret
        print "status=",stat
        time.sleep(1)

In [6]:
def statword(state):
    shb_err_status = 0
    shr_err_status = 0
    
    if state in [0,10]:
        shb_open_status = 0
        shb_close_status = 1
        shr_open_status = 0
        shr_close_status = 1

    elif state==20:
        shb_open_status = 1
        shb_close_status = 0
        shr_open_status = 1
        shr_close_status = 0
    elif state==30:
        shb_open_status = 1
        shb_close_status = 0
        shr_open_status = 0
        shr_close_status = 1
    elif state==40:
        shb_open_status = 0
        shb_close_status = 1
        shr_open_status = 1
        shr_close_status = 0
        
    statword =(1<<6)+ (shb_open_status<<5)+(shb_close_status<<4)+(shb_err_status<<3)+(shr_open_status<<2)+(shr_close_status<<1)+shr_err_status;
    return str(statword)
    

In [7]:
init_state_00 = [('init', ''),
                 ('status', '0'),
                 ('statword', statword(0))]

init_state_10 =  init_state_00 + \
                 [('bia_on', ''),
                 ('status', '10'),
                 ('statword', statword(10))]
    
init_state_20 = init_state_00 + \
                 [('shut_open', ''),
                 ('status', '20'),
                 ('statword', statword(20))]
    
init_state_30 = init_state_00 + \
                 [('blue_open', ''),
                 ('status', '30'),
                 ('statword', statword(30))]
    
init_state_40 = init_state_00 + \
                 [('red_open', ''),
                 ('status', '40'),
                 ('statword', statword(40))]


In [8]:
processSeq(init_state_00)

init
 == 
status
0 == 0
statword
82 == 82


In [23]:
processSeq(init_state_10)

init
 == 
status
0 == 0
statword
82 == 82
bia_on
 == 
status
10 == 10
statword
82 == 82


In [9]:
processSeq(init_state_20)

init
 == 
status
0 == 0
statword
82 == 82
shut_open
 == 
status
20 == 20
statword
100 == 100


In [10]:
processSeq(init_state_30)

init
 == 
status
0 == 0
statword
82 == 82
blue_open
 == 
status
30 == 30
statword
98 == 98


In [11]:
processSeq(init_state_40)

init
 == 
status
0 == 0
statword
82 == 82
red_open
 == 
status
40 == 40
statword
84 == 84


In [8]:
testAll(init_state_00)

cmd= bia_on
ret= 
status= 10
cmd= bia_off
ret= n
status= 0
cmd= shut_open
ret= 
status= 20
cmd= shut_close
ret= n
status= 0
cmd= blue_open
ret= 
status= 30
cmd= blue_close
ret= n
status= 0
cmd= red_open
ret= 
status= 40
cmd= red_close
ret= n
status= 0
cmd= init
ret= 
status= 0


In [9]:
testAll(init_state_10)

cmd= bia_on
ret= n
status= 10
cmd= bia_off
ret= 
status= 0
cmd= shut_open
ret= n
status= 10
cmd= shut_close
ret= n
status= 10
cmd= blue_open
ret= n
status= 10
cmd= blue_close
ret= n
status= 10
cmd= red_open
ret= n
status= 10
cmd= red_close
ret= n
status= 10
cmd= init
ret= 
status= 0


In [10]:
testAll(init_state_20)

cmd= bia_on
ret= n
status= 20
cmd= bia_off
ret= n
status= 20
cmd= shut_open
ret= n
status= 20
cmd= shut_close
ret= 
status= 0
cmd= blue_open
ret= n
status= 20
cmd= blue_close
ret= 
status= 40
cmd= red_open
ret= n
status= 20
cmd= red_close
ret= 
status= 30
cmd= init
ret= 
status= 0


In [11]:
testAll(init_state_30)

cmd= bia_on
ret= n
status= 30
cmd= bia_off
ret= n
status= 30
cmd= shut_open
ret= 
status= 20
cmd= shut_close
ret= 
status= 0
cmd= blue_open
ret= n
status= 30
cmd= blue_close
ret= 
status= 0
cmd= red_open
ret= 
status= 20
cmd= red_close
ret= n
status= 30
cmd= init
ret= 
status= 0


In [15]:
testAll(init_state_40)

cmd= bia_on
ret= n
status= 40
cmd= bia_off
ret= n
status= 40
cmd= shut_open
ret= 
status= 20
cmd= shut_close
ret= 
status= 0
cmd= blue_open
ret= 
status= 20
cmd= blue_close
ret= n
status= 40
cmd= red_open
ret= n
status= 40
cmd= red_close
ret= 
status= 0
cmd= init
ret= 
status= 0


In [20]:
other = ['pulse_on',
        'pulse_off',
        'set_period1500',
        'set_duty1228']

for cmd in other:
    print "period=", dev.sendOneCommand('get_period')
    print "duty=", dev.sendOneCommand('get_duty')
    print "param=", dev.sendOneCommand('get_param')
    print 
    print cmd
    print "ret=", dev.sendOneCommand(cmd)
    
print "period=", dev.sendOneCommand('get_period')
print "duty=", dev.sendOneCommand('get_duty')
print "param=", dev.sendOneCommand('get_param')

period= 1500
duty= 128
param= 0,1500,128

pulse_on
ret= 
period= 1500
duty= 128
param= 1,1500,128

pulse_off
ret= 
period= 1500
duty= 128
param= 0,1500,128

set_period1500
ret= 
period= 1500
duty= 128
param= 0,1500,128

set_duty1228
ret= 
period= 1500
duty= 128
param= 0,1500,128


In [ ]:
try:
    self.checkInterlock(self.shState, self.biaState, cmdStr)
except Exception as e:
    cmd.warn("text='%s switch failed %s'" % (self.name.upper(), formatException(e, sys.exc_info()[2])))
    return True

try:
    reply = self.sendOneCommand(cmdStr, doClose=False, cmd=cmd)
    if reply != "":
        raise Exception("warning %s return %s" % (cmdStr, reply))

except Exception as e:
    cmd.warn("text='%s switch failed %s'" % (self.name.upper(), formatException(e, sys.exc_info()[2])))
    return False

return True

In [ ]:
def warn(func):
    def wrapper(self, *args, **kwargs):
        try:
            func()
        except
        

In [21]:
t = dev.sendOneCommand('get_param')

In [24]:
[int(val) for val in t.split(',')]

[0, 100, 0]

In [ ]:
both_close = 109
both_open = 91
redopen = 93
blueopen = 107




In [101]:
print 91-64

27


In [99]:
dev.sendOneCommand('status')

'30'

In [98]:
dev.sendOneCommand('statword')

'107'

In [97]:
dev.sendOneCommand('blue_open')

'shutter switch timeoutn'

In [35]:
statword = dev.sendOneCommand('statword')
shut_stat = [{0: "close", 1: "open"}, {0: "open", 1: "close"}, {0: "ok", 1: "error"}]

In [26]:
for i, shutter in enumerate(["shr", "shb"]):
        print "%s=%s" % (shutter, ','.join([shut_stat[j % 3][int(statword[j])] for j in range(i * 3, (i + 1) * 3)]))


KeyError: 9

In [36]:
statword

'109'

In [85]:
hex(int(statword))

TypeError: int() argument must be a string or a number, not 'function'

In [78]:
dev.sendOneCommand('status')

'40'

In [47]:
statword

97

In [84]:
hex(int(dev.sendOneCommand('statword')))

Exception: bsh failed to connect socket

In [12]:

shb_err_status = 0
shb_open_status = 1
shb_close_status = 0

shr_err_status = 0
shr_open_status = 0
shr_close_status = 1    

statword =(1<<6)+ (shb_open_status<<5)+(shb_close_status<<4)+(shb_err_status<<3)+(shr_open_status<<2)+(shr_close_status<<1)+shr_err_status;
print hex(statword)
    

0x62


In [51]:
82
100
blue 84
red 98

12

In [66]:

1<<1

2

In [13]:
dev.sendOneCommand('blue_open')

''

In [14]:
dev.sendOneCommand('statword')

'010100'

In [20]:
hex(98)

'0x62'

In [21]:
1<<6

64